In [1]:
import os
from pymilvus import (connections, MilvusClient, utility)
from dotenv import load_dotenv

In [2]:

#/var/tmp/.env
load_dotenv()  # Loads from .env file by default
api_key = os.getenv("GOOGLE_API_KEY")
debug = os.getenv("DEBUG")
pinecone_api_key=os.getenv("PINECONE_API_KEY")
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
MILVUS_USER=os.getenv("MILVUS_USERNAME")
MILVUS_PASSWORD = os.getenv("MILVUS_PASSWORD")
MILVUS_URI = os.getenv("MILVUS_URI")
MILVUS_API_TOKEN = os.getenv("MILVUS_API_TOKEN")

In [4]:
import time
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Milvus
# Removed direct import: from langchain_community.retrievers import MMRRetriever
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from pymilvus import connections, utility
from docx import Document
import os
import pdfplumber # Import pdfplumber

# --- Step 1: Load & Chunk Text ---
#pdf_path = r"C:\Gajanan\data\PrinceCatalogue.pdf"
pdf_path = r"C:\Gajanan\data\imageodf.pdf"
full_text = ""
try:
    # Use pdfplumber to extract text from the PDF
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # Extract text from each page and append to full_text
            page_text = page.extract_text()
            if page_text: # Ensure text was extracted from the page
                full_text += page_text + "\n" # Add a newline between pages
    if not full_text.strip(): # Check if any text was actually extracted
         print("Warning: No text extracted from the PDF. Check if the PDF contains extractable text.")
         full_text = "Placeholder text as no text could be extracted from the PDF." # Use placeholder if extraction fails
    print(full_text)
except FileNotFoundError:
    print(f"Error: PDF file not found at the specified path: {pdf_path}")
    full_text = "Placeholder text because the PDF file was not found. Please replace with actual PDF content extraction."
except Exception as e:
    print(f"Error processing PDF file {pdf_path}: {e}")
    full_text = "Placeholder text due to an error during PDF processing."











Placeholder text as no text could be extracted from the PDF.


In [57]:
print(len(full_text))

591891


In [11]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"


In [ ]:
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io


def extract_text_from_image_pdf(pdf_path, output_txt_path=None, dpi=300, lang='eng', verbose=True):
    """
    Extract text from an image-based PDF using OCR (Tesseract + PyMuPDF).
    
    Args:
        pdf_path (str): Path to the input PDF file.
        output_txt_path (str, optional): Path to save the extracted text. If None, won't save.
        dpi (int): Dots per inch for rendering. Higher = better OCR.
        lang (str): Language(s) for Tesseract OCR. e.g., 'eng', 'eng+deu'
        verbose (bool): Print progress messages.

    Returns:
        str: Extracted text from the PDF.
    """
    doc = fitz.open(pdf_path)
    all_text = ""

    for page_num in range(len(doc)):
        if verbose:
            print(f"OCR processing page {page_num + 1}/{len(doc)}...")
        page = doc.load_page(page_num)
        pix = page.get_pixmap(dpi=dpi)
        img = Image.open(io.BytesIO(pix.tobytes("png")))

        # Optional preprocessing (e.g., grayscale)
        img = img.convert("L")

        # OCR
        text = pytesseract.image_to_string(img, lang=lang)
        all_text += f"\n\n--- Page {page_num + 1} ---\n{text}"

    doc.close()

    if output_txt_path:
        with open(output_txt_path, 'w', encoding='utf-8') as f:
            f.write(all_text)

    return all_text


In [14]:
full_text = extract_text_from_image_pdf(
    pdf_path="C:\Gajanan\data\imageodf.pdf",
    output_txt_path="output_text.txt",
    dpi=300,
    lang='eng',
    verbose=True
)


<>:2: SyntaxWarning: invalid escape sequence '\G'
<>:2: SyntaxWarning: invalid escape sequence '\G'
C:\Users\Admin\AppData\Local\Temp\ipykernel_8660\3183012486.py:2: SyntaxWarning: invalid escape sequence '\G'
  pdf_path="C:\Gajanan\data\imageodf.pdf",


OCR processing page 1/6...
OCR processing page 2/6...
OCR processing page 3/6...
OCR processing page 4/6...
OCR processing page 5/6...
OCR processing page 6/6...


In [15]:
print(full_text)



--- Page 1 ---
Intelligent Investment

India Market Monitor
O4 2024
Residential

REPORT ———————_ CBRE RESEARCH

January 2025

“) PR SEAR MERIT TET
py My if Ww vie & a) > 2



--- Page 2 ---
Residential

India's residential real estate market demonstrated positive activity throughout 2024,
culminating in a strong final quarter performance accentuated by seasonal festive
demand. This surge propelled annual sales to surpass 300,000 housing units for the
second consecutive year. Developers responded to this market momentum by
introducing over 280,000 new residential units during the year.

[e) [e)
Th 40%
Q-o-Q growth in housing unit Share of mid-end projects in sales in 2024,
sales in Q4 2024 followed by high-end and affordable projects*

64% 66%

Cumulative share of Mumbai, Pune, and Total share of Mumbai, Pune, and Delhi-NCR
Bengaluru in sales in 2024 in sales in Q4 2024

10% 63%

Growth in new unit launches on Q-o-Q basis Combined share of Mumbai, Pune, and
in Q4 2024 Hyderabad in uni

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_text(full_text)
sum_chunk_total_chars = 0
for chunk in chunks:
    #print(len(chunk))
    sum_chunk_total_chars +=len(chunk)

print(f"Total chunks: {len(chunks)}")

print("sum_chunk_total_chars",sum_chunk_total_chars)
# --- Step 2: Embedding ---
# Ensure 'YOUR_OPENAI_API_KEY' is replaced with an actual valid key or an environment variable
openai_api_key = OPENAI_KEY# Use environment variable or replace
if openai_api_key == OPENAI_KEY:
    print("Warning: Replace 'YOUR_OPENAI_API_KEY' with your actual key or set the OPENAI_API_KEY environment variable.")

embeddings = OpenAIEmbeddings(model="text-embedding-3-small", api_key=openai_api_key,disallowed_special=()) # Use the variable


Total chunks: 11
sum_chunk_total_chars 8819


In [21]:
# --- Step 3: Connect to Milvus and Store Data ---
# Helper to create Milvus index
def store_milvus(chunks, index_name, metric_type, index_type):
    if utility.has_collection(index_name):
        utility.drop_collection(index_name)
    # Ensure the chunks are not empty before creating the collection
    if not chunks:
        print(f"Warning: No chunks to store for index '{index_name}'. Skipping collection creation.")
        return None

    return Milvus.from_texts(
        chunks,
        embedding=embeddings,
        collection_name=index_name,
        connection_args={"uri": "https://in03-c8ed7ba112ba348.serverless.gcp-us-west1.cloud.zilliz.com",
        "token": MILVUS_API_TOKEN},
        index_params={"metric_type": metric_type, "index_type": index_type})


In [65]:
print(Milvus.__doc__)


.. deprecated:: 0.2.0 Use ``:class:`~langchain_milvus.MilvusVectorStore``` instead. It will not be removed until langchain-community==1.0.

`Milvus` vector store.

You need to install `pymilvus` and run Milvus.

See the following documentation for how to run a Milvus instance:
https://milvus.io/docs/install_standalone-docker.md

If looking for a hosted Milvus, take a look at this documentation:
https://zilliz.com/cloud and make use of the Zilliz vectorstore found in
this project.

IF USING L2/IP metric, IT IS HIGHLY SUGGESTED TO NORMALIZE YOUR DATA.

Args:
    embedding_function (Embeddings): Function used to embed the text.
    collection_name (str): Which Milvus collection to use. Defaults to
        "LangChainCollection".
    collection_description (str): The description of the collection. Defaults to
        "".
    collection_properties (Optional[dict[str, any]]): The collection properties.
        Defaults to None.
        If set, will override collection existing properties.
   

In [22]:
milvus_flat = None # Initialize vectorstore variables
milvus_hnsw = None
milvus_ivf = None
milvus_connected = False # Flag to indicate if Milvus connection was successful

# Ensure Milvus server is running and accessible at localhost:19530
try:
    #connections.connect("default", host="localhost", port="19530")
    # Connect to Zilliz cloud using endpoint URI and API key TOKEN.
    #CLUSTER_ENDPOINT="https://in03-xxxx.api.gcp-us-west1.zillizcloud.com:443"
    #CLUSTER_ENDPOINT="https://in03-8bc9fd463236b1a.api.gcp-us-west1.zillizcloud.com:443"
    CLUSTER_ENDPOINT = "https://in03-c8ed7ba112ba348.serverless.gcp-us-west1.cloud.zilliz.com"

    connections.connect(
    alias='default',
    uri=CLUSTER_ENDPOINT,
    token=MILVUS_API_TOKEN

    )
    print("Connected. Existing collections:", utility.list_collections())

    # Check if the server is ready and get collection name.
    print(f"Type of server: {utility.get_server_version()}")
    print("Connected to Milvus")
    milvus_connected = True # Set flag to True on successful connection

    # Create three indexes ONLY if co
    # Create three indexes ONLY if connection is successful
    #milvus_flat = store_milvus(chunks,"flat_index", "L2", IndexType.FLAT)
    milvus_hnsw = store_milvus(chunks, "hnsw_index", "COSINE", "HNSW")
    milvus_ivf = store_milvus(chunks, "ivf_index", "COSINE", "IVF_FLAT")
except Exception as e:
    print(f"Error connecting to Milvus or storing data: {e}")
    # The vectorstore variables will remain None if connection fails

Connected. Existing collections: ['ivf_index', 'hnsw_index', 'genai_collection', 'flat_index', 'hnsw_index_image']
Type of server: Zilliz Cloud Vector Database(Compatible with Milvus 2.4)
Connected to Milvus


In [23]:
# --- Step 4: Retriever + Timer + Accuracy ---
def test_retriever(name, vectorstore, query):
    if vectorstore is None: # Handle cases where collection wasn't created
        print(f"Skipping {name} retriever test as the collection was not created.")
        return []

    # Use search_type argument for different retriever types
    # Ensure the vectorstore object has the as_retriever method
    if hasattr(vectorstore, 'as_retriever'):
        if name == "MMR":
            retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 5, "lambda_mult": 0.5})
        else:
            retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

        start = time.time()
        # Ensure retriever returns list of Documents; handle potential errors
        try:
            docs = retriever.get_relevant_documents(query)
        except Exception as e:
            print(f"Error during {name} retrieval: {e}")
            docs = []
            return docs

        end = time.time()
        print(f"{name} Retriever Time: {end - start:.4f}s")
        # Calculate accuracy if docs are not empty
        # Note: Simple substring match for accuracy is basic;
        # more sophisticated methods are needed for robust evaluation.
        if docs:
            # Ensure document objects have 'page_content' attribute
            hits = sum(1 for d in docs if hasattr(d, 'page_content') and query.lower() in d.page_content.lower())
            accuracy = hits / len(docs)
            print(f"{name} Accuracy: {accuracy:.2%}")
        else:
             print(f"{name} Accuracy: N/A (no documents retrieved)")

        return docs
    else:
        print(f"Error: Vectorstore object for {name} does not have 'as_retriever' method.")
        return []


In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from docx import Document

def run_prompt_tuning_pipeline(query, milvus_flat, milvus_hnsw, milvus_ivf, milvus_connected, openai_api_key):
    docs_flat, docs_hnsw, docs_mmr = [], [], []
    
    if milvus_connected:
        docs_flat = test_retriever("Flat", milvus_flat, query)
        docs_hnsw = test_retriever("HNSW", milvus_hnsw, query)
        print("hnsw Top Docs:", docs_hnsw)
        print("hnsw context",docs_hnsw[1])
        docs_mmr = test_retriever("MMR", milvus_ivf, query)
        print("MMR Top Docs:", docs_mmr)
        print("mmr ivf context",docs_mmr[1])
        print("Skipping retriever tests as Milvus connection failed.")
    
    prompt = PromptTemplate.from_template("""
You are an AI assistant. Use the following retrieved context to answer the question.

Context:
{context}

Question:
{question}
""")
    
    llm = ChatOpenAI(model="gpt-4o", temperature=0.2, api_key=openai_api_key)
    output = "Could not generate answer due to an error or Milvus connection failure."

    if milvus_ivf is not None and milvus_connected:
        try:
            retriever = milvus_ivf.as_retriever(search_type="mmr", search_kwargs={"k": 5, "lambda_mult": 0.5})
            chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type_kwargs={"prompt": prompt})
            output = chain.run(query)
            print("Answer:\n", output)
        except Exception as e:
            print(f"Error running RetrievalQA chain with MMR: {e}")
            output = f"Could not generate answer due to an error: {e}"
    else:
        print("Milvus IVF collection was not initialized or Milvus connection failed.")
        output = "Could not create RetrievalQA chain because the Milvus IVF collection was not initialized or Milvus connection failed."

    # Export to DOCX if successful
    if not output.startswith("Could not"):
        try:
            doc = Document()
            doc.add_heading("LLM Answer", level=1)
            doc.add_paragraph(output)
            doc.save("LLM_Answer_Output.docx")
            print("Output saved to LLM_Answer_Output.docx")
        except Exception as e:
            print(f"Error saving output to DOCX: {e}")
    else:
        print("DOCX file not saved due to an error in generating the answer.")

    return {
        "flat_docs": docs_flat,
        "hnsw_docs": docs_hnsw,
        "mmr_docs": docs_mmr,
        "answer": output
    }


In [25]:
query = "Give info about indias residensial market?"
results = run_prompt_tuning_pipeline(
    query=query,
    milvus_flat=milvus_flat,
    milvus_hnsw=milvus_hnsw,
    milvus_ivf=milvus_ivf,
    milvus_connected=milvus_connected,
    openai_api_key=OPENAI_KEY
)


Skipping Flat retriever test as the collection was not created.


C:\Users\Admin\AppData\Local\Temp\ipykernel_8660\108715618.py:18: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


HNSW Retriever Time: 0.9569s
HNSW Accuracy: 0.00%
hnsw Top Docs: [Document(metadata={'pk': 458312173146087084}, page_content="--- Page 1 ---\nIntelligent Investment\n\nIndia Market Monitor\nO4 2024\nResidential\n\nREPORT ———————_ CBRE RESEARCH\n\nJanuary 2025\n\n“) PR SEAR MERIT TET\npy My if Ww vie & a) > 2\n\n\n\n--- Page 2 ---\nResidential\n\nIndia's residential real estate market demonstrated positive activity throughout 2024,\nculminating in a strong final quarter performance accentuated by seasonal festive\ndemand. This surge propelled annual sales to surpass 300,000 housing units for the\nsecond consecutive year. Developers responded to this market momentum by\nintroducing over 280,000 new residential units during the year.\n\n[e) [e)\nTh 40%\nQ-o-Q growth in housing unit Share of mid-end projects in sales in 2024,\nsales in Q4 2024 followed by high-end and affordable projects*\n\n64% 66%\n\nCumulative share of Mumbai, Pune, and Total share of Mumbai, Pune, and Delhi-NCR\nBengal

C:\Users\Admin\AppData\Local\Temp\ipykernel_8660\515078519.py:29: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o", temperature=0.2, api_key=openai_api_key)
C:\Users\Admin\AppData\Local\Temp\ipykernel_8660\515078519.py:36: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = chain.run(query)


Answer:
 India's residential real estate market showed positive activity throughout 2024, with a strong performance in the final quarter driven by seasonal festive demand. This led to annual sales surpassing 300,000 housing units for the second consecutive year. Developers responded by introducing over 280,000 new residential units during the year. The market is characterized by strong fundamentals and robust end-user demand, with expectations for steady sales and new launches. However, elevated capital values and an uncertain global economic landscape may lead homebuyers to adopt a wait-and-watch approach.

The market saw a significant share of sales in mid-end projects, followed by high-end and affordable projects. Mumbai, Pune, and Bengaluru accounted for a large portion of sales in 2024. There is sustained demand for high-end and premium residential categories, particularly those priced between INR 1.5 to 2 crore and INR 2 to 4 crore. Markets like Noida, Bengaluru, Pune, and Chenna

In [27]:
import pandas as pd

def write_results_to_csv_df(results, query,filename="results_output.csv"):
    # Normalize list values for clean storage
    formatted_results = {}

    for key, value in results.items():
        if isinstance(value, list):
            # Convert list of items to a single string (join by separator)
            formatted_results[key] = ["\n---\n".join(str(v) for v in value)]
        else:
            formatted_results[key] = [str(value)]  # Wrap in list to make it a row

    # Create a DataFrame with one row and keys as columns
    df = pd.DataFrame.from_dict(formatted_results)
    df["query"]=query
    # Save to CSV
    df.to_csv(filename, index=False, encoding="utf-8")
    print(f"Results saved to {filename}")


In [28]:
write_results_to_csv_df(results,query, filename="results_output_image_q1.csv")

Results saved to results_output_image_q1.csv


In [30]:
results = run_prompt_tuning_pipeline(
    query="Give status in different states in india market?",
    milvus_flat=milvus_flat,
    milvus_hnsw=milvus_hnsw,
    milvus_ivf=milvus_ivf,
    milvus_connected=milvus_connected,
    openai_api_key=OPENAI_KEY
)


Skipping Flat retriever test as the collection was not created.
HNSW Retriever Time: 0.9566s
HNSW Accuracy: 0.00%
hnsw Top Docs: [Document(metadata={'pk': 458312173146087084}, page_content="--- Page 1 ---\nIntelligent Investment\n\nIndia Market Monitor\nO4 2024\nResidential\n\nREPORT ———————_ CBRE RESEARCH\n\nJanuary 2025\n\n“) PR SEAR MERIT TET\npy My if Ww vie & a) > 2\n\n\n\n--- Page 2 ---\nResidential\n\nIndia's residential real estate market demonstrated positive activity throughout 2024,\nculminating in a strong final quarter performance accentuated by seasonal festive\ndemand. This surge propelled annual sales to surpass 300,000 housing units for the\nsecond consecutive year. Developers responded to this market momentum by\nintroducing over 280,000 new residential units during the year.\n\n[e) [e)\nTh 40%\nQ-o-Q growth in housing unit Share of mid-end projects in sales in 2024,\nsales in Q4 2024 followed by high-end and affordable projects*\n\n64% 66%\n\nCumulative share of Mumb

In [31]:
query="Give status in different states in india market?"

In [32]:
write_results_to_csv_df(results, query,filename="results_output_image_q2.csv")

Results saved to results_output_image_q2.csv


In [35]:
query="What is avg sales in Delhi?"

In [37]:
results = run_prompt_tuning_pipeline(
   query="What is avg sales in Delhi?",
    milvus_flat=milvus_flat,
    milvus_hnsw=milvus_hnsw,
    milvus_ivf=milvus_ivf,
    milvus_connected=milvus_connected,
    openai_api_key=OPENAI_KEY
)


Skipping Flat retriever test as the collection was not created.
HNSW Retriever Time: 0.6430s
HNSW Accuracy: 0.00%
hnsw Top Docs: [Document(metadata={'pk': 458312173146087085}, page_content='64% 66%\n\nCumulative share of Mumbai, Pune, and Total share of Mumbai, Pune, and Delhi-NCR\nBengaluru in sales in 2024 in sales in Q4 2024\n\n10% 63%\n\nGrowth in new unit launches on Q-o-Q basis Combined share of Mumbai, Pune, and\nin Q4 2024 Hyderabad in unit launches in 2024\n\n60% "Ne e - Definition of segments as per ticket size (INR): Aff peo :\n\n- Up to 45 lakh; Mic = h-er\n\nCr; Premium - 2 Cr to 4 Cr, and\n\nCumulative share of Mumbai, Pune, and\nBengaluru in unit launches in Q4 2024\n\n“AYA\nihe Ne\n\nra\n\n©2025 CBRE;INC.\n\n\n\n--- Page 3 ---\nMarket Performance in Q4 2024\n\nIndia Market Monitor\n\n~302,900\n\nSales in 2024\n\n~281,600\n\nUnit launches in 2024\n\n~/9,300\n\nSales in Q4 2024\n\n~67,800\n\nUnit launches in Q4 2024\n\nUnit la\n\nnches\n\nin Q4 2024\n\nQ4 2024 capital val

In [38]:
write_results_to_csv_df(results, query,filename="results_output_image_q3.csv")

Results saved to results_output_image_q3.csv
